# 9장. 문서 요약 (Text Summarization)

# 9-0 데이터 준비

In [1]:
import requests 
from bs4 import BeautifulSoup
import re

def cleansing(soup):
    body_content = soup.select_one("#articleBodyContents")
    content = body_content.get_text()
    for strong in body_content.select("strong"):
        content = content.replace(strong.get_text(), "")

    for td in body_content.select("td"):
        content = content.replace(td.get_text(), "")
    
    content = re.sub(r"\[[가-힣 ]+\]", "", content)
    start_pos = re.search(r"[가-힣]{2,4}\s\(?\w+@\w+\.\w+(.\w+)?\)", content).start()
    content = content[:start_pos]
    content = content.replace("\n", "").replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")


    return content

def get_news_by_url(url):
  headers={"user-agent":"Mozilla/5.0"}
  res = requests.get(url, headers=headers)
  soup = BeautifulSoup(res.content, "html.parser")
  content = cleansing(soup)
  return content

doc = get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108')
print(doc[:50])
print(doc[-50:])


 국가 차원의 빅데이터 활용 시대가 열린다. 새로운 산업 창출과 기존 산업의 변화에 이르는
 혁신하고 기업의 경쟁력을 한 단계 제고할 수 있도록 정책적 역량을 집중하겠다”고 밝혔다.


## Mecab 설치 (필요시)

# 9-1 Luhn Summarizer

http://courses.ischool.berkeley.edu/i256/f06/papers/luhn58.pdf

Hans Peter Luhn

https://en.wikipedia.org/wiki/Hans_Peter_Luhn

 Hans Peter Luhn은 공학과 정보과학에서의 개척 작업으로 "정보 검색의 아버지"로 알려져 있다. 그는 표제어가 문맥에 포함된 채 배열된 색인(KWIC : keyword-in-context) 개발, 정보 선택 제공(SDI), 완전 텍스트 프로세싱, 자동 발췌(요약), 단어 시소로스의 최초 현대식 사용으로 신뢰를 얻었다. 오늘날 파생된 지식 대부분에는 KWIC 색인이 있으며 과학의 모든 분야에 SDI시스템이 있다. 


 Luhn은 1896년 7월 1일 독일 바르멘에서 태어났다. 아버지가 그 당시 유명한 인쇄업자였으므로, 스위스에서 인쇄업을 배웠다. 어렸을 때무터 그는 창조성이 뛰어난 재능을 보였으며, 기술적 문제, 물리학, 통계학에 관심을 보였다. 그러나 1차 세계대전으로 독일군 통신장교로 복역(1915년1917년)하면서 프랑스 터키 루마니아 불가리아 등지로 옮겨 다녀야만 했다. 1차 대전 이후 그는 스위스의 Gallen 교회 Schweizwrische handels Hochschule로 돌아와 기술, 물리학, 회계분야의 수업을 들었다. 그 후, Luhn은 그리스에서 못다한 공부에 전념했으며, 더블부기기계(Duble-Entry Bookeeping Machine : 카드 대장에 대변기입과 차변 기입을 기록할 수 있는 것)를 발명하였다. 그는 또 Hollerith tabulating/recording machine에 정통했고, 천공카드에서 문자 숫자를 나타내는 장치의 사용을 증가시키게 했다. 1920년부터 1930년까지는 10개의 특허권을 획득하여 그의 탁월한 능력을 보여주었다. 그것들중에 루노메터(Lunometer : 직물의 실길이를 계산하는데 쓰이는 장치)는 지금도 사용되고 있다.


 1920년까지 그는 직물 공장에서 일하기 시작했다. 그는 직물 공장의 사업확장을 위해 뉴잉글랜드에서 1924년 미국으로 가게 되었다. 그러나 회사가 곧 파산하였고 Luhn은 직장없이 뉴욕에 남게 되었다. 그는 은행에서 일을 하였고 곧 뉴욕 월스트리트에 소재한 국제어음은행(International Acceptance Bank)에서 재정담당관으로 승진하였다. 


 1933년 Luhn은 자사인 공학회사 H.P. Luhn & Association을 설립하였고 8년간 자문기술자로 일했다. 1941년 Luhn은 IBM에서 수석 연구기술자로 참여하였고 이후에 정보검색연구 관리자로 일했다. Luhn이 IBM에서 새로운 아이디어를 지속적으로 내놓고 문제를 다르게 접근하여 주목을 받는 동안, 다른 기술자들에게 고차원적인 창조를 하도록 자극하면서 그들의 촉매제 역할을 하여 신뢰를 얻었다

In [ ]:
doc

In [ ]:
import nltk
nltk.download('punkt')

### 1) 토큰화

In [2]:
from nltk.tokenize import sent_tokenize
from kss import split_sentences
from konlpy.tag import Mecab
mecab = Mecab()
# 문장 분리
def get_sentences(txt):
  return split_sentences(txt)

# 토큰화 
def get_words(txt):
  return [token[0] for token in mecab.pos(txt) if token[1][0] == 'N' and len(token[0]) > 1]

### 2) 중요 단어 결정

In [3]:
# 단어(토큰)의 가중치 계산 및 범위에 포함되는 토큰 식별 
def get_keywords(word_list , min_ratio=0.001, max_ratio=0.5) :
    assert (min_ratio < 1 and max_ratio < 1)
    
    # 토큰별로 빈도수 카운팅
    count_dict = {}
    for word in word_list:
        # if word in count_dict.keys():
        #     count_dict[word] += 1
        # else:
        #     count_dict[word] = 1
        count_dict.setdefault(word, 0)
        count_dict[word] += 1

    # 분석 문서의 총 토큰수 대비 해당 토큰의 빈도 비율
    keywords = set()
    for word, cnt in count_dict.items():
       word_percentage = cnt/len(word_list)

       # 사전 정의한 비율내에 포함 된 경우 키워드에 추가
       if word_percentage >= min_ratio and word_percentage <= max_ratio:
           keywords.add(word)
        
    return keywords

get_keywords(['바나나','사과','바나나','바나나','포도'])

{'사과', '포도'}

### 3) 문장 중요도 계산

In [4]:
# 문장의 가중치 계산
def get_sentence_weight (sentence , keywords):
    tokens = sentence.split(" ")
    window_start, window_end = 0, -1

    # 문장내에서 윈도 시작 위치 탐색
    # 범위내 속한 키워드가 등장하는 첫번째 위치 계산
    for i in range(len(tokens)):
        if tokens[i] in keywords:
            window_start = i
            break
    
    
    # 문장내에서 윈도 종료 위치 탐색
    # 범위내 속한 키워드가 등장하는 마지막 위치 계산
    for i in range(len(tokens) - 1 ,0 ,-1):
        if tokens[i] in keywords:
            window_end = i
            break
    
    # 윈도의 시작위치가 종료위치보다 큰경우 => 분석할 단어(토큰)가 없는 경우 종료
    if window_start > window_end:
        return 0

    # 윈도 크기 계산
    window_size = window_end - window_start + 1
    
    # 분석 대상 문장 중 범위(0.001 ~ 0.5)에 포함된 토큰 개수 카운팅
    keyword_cnt = 0
    for w in tokens[window_start : window_start + window_size]:
        if w in keywords:
            keyword_cnt += 1
    
    # (분석 대상 문장 중 범위(0.001 ~ 0.5)에 포함된 토큰 개수) / 윈도사이즈
    return keyword_cnt * keyword_cnt * 1.0 / (window_size)

### 4) 문서 요약

In [5]:
# 문서 요약
def summarize(content ,max_no_of_sentences = 10):
    
    
    # 단어(토큰) 분리
    word_list = get_words(content)
    
    # 단어(토큰) 가중치 계산 및 범위 내 포함 단어(토큰) 추출
    keywords = get_keywords(word_list)

    # 문장별 가중치 계산
    sentence_list = get_sentences(content)
    sentence_weight = []

    for sentence in sentence_list:
        sentence_weight.append((get_sentence_weight(sentence, keywords), sentence))
           
    # 문장별 가중치 역순 계산
    sentence_weight.sort(reverse=True)
    
    return [ weight[1] for weight in sentence_weight[:max_no_of_sentences]]

In [6]:
li = summarize (doc ,  3)
for s in li :
    print(s)
    print("\n")

22일 과학기술정보통신부는 서울 중구 대한상공회의소에서 데이터 생태계 조성과 혁신 성장의 기반 마련을 위한 ‘빅데이터 플랫폼 및 센터’ 출범식 행사를 개최했다.


특히 공공과 민간 사이 데이터 파일형식 등이 달라 호환이 제대로 이뤄지지 못한 문제를 해소하기 위해 개방형 표준을 적용하고, 품질관리기준도 마련해 운영한다.


금융 플랫폼의 경우 소상공인 신용평가 고도화 등을 통해 금융 취약 계층 대상 중금리 대출이자를 2%p 절감해 연간 1조원의 신규대출을 창출할 전망이다.






---



# 9-2 Textrank

![대체 텍스트](https://www.researchgate.net/profile/Khushboo_Thakkar3/publication/232645575/figure/fig1/AS:575720050573312@1514273764062/Sample-graph-build-for-sentence-extraction.png)

## 2.1 TextRank 직접 구현하기 (Matrix 활용)

### 1) 자카드 유사도

In [ ]:
Text = "딸기 바나나 사과 파인애플. 바나나 사과 딸기 포도. 복숭아 수박. 파인애플 사과 딸기 바나나."

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()
# 문장간 유사도 측정 (자카드 유사도 사용)
def sentence_similarity(sentence1, sentence2):
  sentence1 = [ for token in mecab.pos(sentence1) if ]
  pass
  

sentence_similarity('나는 치킨을 좋아해','나는 치킨을 싫어해')

### 2) 그래프 생성

In [ ]:
 import numpy as np
 def buildMatrix(sentences):
    
    # 문장별로 그래프 edge를 Matrix 형태로 생성
    
    # normalize 
    
     
    return 

Text = "딸기 바나나 사과 파인애플 수박. 바나나 사과 딸기 포도. 복숭아 수박. 파인애플 사과 딸기 바나나."
buildMatrix(sent_tokenize(Text))    

### 3) 문장 중요도 계산

In [ ]:
def scoring(A, P, eps=0.0001, d=0.85, max_iter = 50):
  pass

### 4) 문서 요약

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
from nltk.tokenize import sent_tokenize

def summarize(text, n=10):
  pass

In [ ]:
summary = summarize(Text, 3)

for sent in summary :
  print(sent)



---



## 2.2 TextRank 직접 구현하기 (Graph 활용)

In [ ]:
Text = "딸기 바나나 사과 딸기 파인애플. 바나나 사과 딸기. 복숭아 파인애플. 파인애플 사과 딸기 바나나."

### 2) 토큰화

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

In [ ]:
def sentences(text):
    pass

### 3) 자카드 유사도

In [ ]:
from konlpy.tag import Mecab

# 문장간 유사도 측정 (자카드 유사도 사용)
def sentence_similarity(sentence1, sentence2):
  pass 

sentence_similarity('나는 치킨을 좋아해','나는 치킨을 싫어해')

In [ ]:
def connect(nodes):
    pass

### 3) 그래프 생성

In [ ]:
import networkx as nx

def rank(nodes,edges):
    graph=nx.diamond_graph()
    graph.clear() 
    graph.add_nodes_from(nodes)
    graph.add_weighted_edges_from(edges)

    return nx.pagerank(graph)

### 4) 문서 요약

In [ ]:
def summarize(text,num_summaries=3):
    pass


In [ ]:
summary=summarize(Text, 3)

for sent in summary :
  print(sent)

## 2.3 gensim을 활용한 요약

In [ ]:
from gensim.summarization.summarizer import summarize
summarize(doc).split('\n')



---

